In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.optimize import fmin_bfgs, fmin_cg
np.set_printoptions(precision=2, linewidth=120)

/Users/yan/.miniconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


Immediately have to address this issue: sampling Qval needs to be Haar-uniform.

In [42]:
PDIAG = np.zeros((9, 9))
for esi in np.eye(3):
    one = np.kron(esi, esi)
    PDIAG = PDIAG + np.outer(one, one)
PDIAG = PDIAG.astype(np.int)

def testFAC(M):
    l = np.linalg.svd(M)[1]
    t1 = (1 - l[2]) >= np.abs(l[0] - l[1])
    t2 = (1 + l[2]) >= np.abs(l[0] + l[1])
    return t1 & t2

def Ls(d=0.1):
    L1 = np.array([[np.cos(d), -np.sin(d), 0],
                   [np.sin(d), np.cos(d), 0],
                   [0, 0, 1]])
    L2 = np.roll(np.roll(L1, 1, axis=0), 1, axis=1)
    L3 = np.roll(np.roll(L2, 1, axis=0), 1, axis=1)
    return L1, L2, L3

def PLSX(d=0.1):
    L1, L2, L3 = Ls(d)
    
    LL1 = np.kron(L1, L1.T)
    LL2 = np.kron(L2, L2.T)
    LL3 = np.kron(L3, L3.T)

    PL1 = np.dot(np.dot(LL1.T, PDIAG), LL1)
    PL2 = np.dot(np.dot(LL2.T, PDIAG), LL2)
    PL3 = np.dot(np.dot(LL3.T, PDIAG), LL3)

    X = np.r_[PL1, PL2, PL3]
    return X

def C1(m, data):
    M = np.array([
                [m[0], m[3], m[4]],
                [m[3], m[1], m[5]],
                [m[4], m[5], m[2]]
            ])
    mvec = np.reshape(M, (9,1))
    return np.linalg.norm(np.dot(PLSX(), mvec) - data) ** 2

def C2(m, data):
    mvec = np.array([
                [m[0], m[3], m[4]],
                [m[3], m[1], m[5]],
                [m[4], m[5], m[2]]
            ])
    M = np.reshape(mvec, (3,3))
    t = 1.0 - testFAC(M).astype(np.int)
    return t

def cost(m, data):
    return C1(m, data) + 1000.0*C2(m, data)

def tomogn(Mval, n):
    mval = 0.5 * (1 - np.diag(Mval))
    T = np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0]])
    Tinv = np.array([[-0.5, 0.5, 0.5], [0.5, -0.5, 0.5], [0.5, 0.5, -0.5]])
    pval = np.dot(Tinv, mval)
    pval = np.r_[1-np.sum(pval), pval]
    e = np.random.multinomial(n, pval)
    p = e / float(n)
    Mhat = np.diag(1.0 - 2.0 * np.dot(T, p[1:]))
    return Mhat

def gatherdata(Mval, n, d=0.1):
    L1, L2, L3 = Ls(d)
    Mval1 = np.dot(np.dot(L1, Mval), L1.T)
    Mval2 = np.dot(np.dot(L2, Mval), L2.T)
    Mval3 = np.dot(np.dot(L3, Mval), L3.T)
    mhat1 = np.reshape(tomogn(Mval1, n), (9,1))
    mhat2 = np.reshape(tomogn(Mval2, n), (9,1))
    mhat3 = np.reshape(tomogn(Mval3, n), (9,1))
    data = np.r_[mhat1, mhat2, mhat3].T
    return data

In [43]:
Qval = np.linalg.qr(np.random.randn(3,3))[0]
Sval = np.diag([0.95, 0.9, 0.85])
Mval = np.dot(np.dot(Qval, Sval), Qval.T)

In [44]:
Mval

array([[ 0.94, -0.02,  0.02],
       [-0.02,  0.86, -0.01],
       [ 0.02, -0.01,  0.91]])

In [45]:
minit = np.random.random((6,1))

In [46]:
data = gatherdata(Mval, 1000000, d=0.5)

In [47]:
mopt = fmin_cg(lambda m: cost(m, data), np.random.random((9,1)), disp=True)

Optimization terminated successfully.
         Current function value: 1175.882164
         Iterations: 2
         Function evaluations: 55
         Gradient evaluations: 5


In [30]:
Mopt = np.reshape(mopt, (3, 3))

In [33]:
Mopt

array([[ 0.87,  0.18,  0.79],
       [ 0.09,  0.93,  0.38],
       [ 0.72,  0.39,  0.9 ]])

In [34]:
testFAC(Mopt)

False

In [32]:
Mval

array([[ 0.87,  0.01, -0.02],
       [ 0.01,  0.94,  0.02],
       [-0.02,  0.02,  0.88]])

In [ ]:
ncycles = []
dist = []
svd_dist = []
Qnormdiffs = []
FACs = []
ds = []
for d in 0.25*np.pi*np.logspace(0, -3, 4):
    for trial in range(1000):
        Qval = np.linalg.qr(np.random.randn(3,3))[0]
        Sval = np.diag([0.95, 0.9, 0.85])
        Mval = np.dot(np.dot(Qval, Sval), Qval.T)
        for n in np.logspace(1, 6, 30):
            Mhat = perfecttomog(Mval)
            Qhat = np.linalg.svd(Mhat)[0]
            Mhatvals = np.linalg.svd(Mhat)[1]
            Mvalvals = np.linalg.svd(Mval)[1]
            dist.append(np.linalg.norm(Mhat - Mval))
            ncycles.append(n)
            svd_dist.append(np.linalg.norm(Mhatvals - Mvalvals))
            ds.append(d)
            Qnormdiffs.append(np.linalg.norm(np.dot(Qhat.T, Qval) - np.eye(3)))
            FACs.append(testFAC(Mhat))

df = pd.DataFrame({"n": ncycles, "dist": svd_dist, "d": ds, "FAC": FACs})
v = df.pivot_table(index="d", columns="n", values="dist", aggfunc=np.mean)
s = df.pivot_table(index="d", columns="n", values="dist", aggfunc=np.std)
dss = df["d"].unique()

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
for idx, d in enumerate(dss):
    t = v.loc[d, :].index.values
    y = v.loc[d, :].values
    e = s.loc[d, :].values
    ax.semilogx(t, y, c=sns.color_palette()[idx], label=d)
    ax.fill_between(t, y-e, y+e, color=sns.color_palette()[idx], alpha=0.25)
plt.legend(frameon=True)
plt.ylabel("Norm distance between $M_{hat}$ and $M_{val}$")
plt.xlabel("Number of error correction cycles")

In [ ]:
qopt = fmin_cg(lambda q: cost(q, S, X),
                 np.random.random((9,1)),
                 disp=False)